In [1]:
import random # Choosing the random response
import json
import pickle # for serialization
import numpy as np
import nltk
from nltk.stem import WordNetLemmatizer # to reduce to stem word
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD, Adam
import torch


In [2]:
!pip install nltk


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 8.1 MB/s 
     |████████████████████████████████| 4.2 MB 45.9 MB/s 
     |████████████████████████████████| 1.2 MB 55.1 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 6.6 MB 41.7 MB/s 
     |████████████████████████████████| 596 kB 45.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=1f2af8cdec43cf307a834aac1f2b44a626d2b2e1668c3945232bf723058ae2a2
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [5]:
with (open('data.json')) as f:
    intents = json.load(f)

documents = []
classes = []
words = []

ignore_letters = ['?', '!', '.', ',']


# New Section

In [6]:
from tensorflow.keras.models import Sequential
for intent in intents['intents']:
  for pattern in intent['patterns']:
    # print(pattern, intent['tag'])
    pattern = pattern.lower()
    pattern_embedding = model.encode(pattern, convert_to_tensor = True)
    # print(len(pattern_embedding.tolist()))
    # print(pattern_embedding.shape)
    documents.append((pattern_embedding.tolist(), intent['tag']))  # documents conatins training data -> (phrase, class) - (["How","are","you"], "greetings")
    if intent['tag'] not in classes:
      classes.append(intent['tag'])
print('classes =',classes)
print(pattern_embedding.shape)      # Features in the embedding - 384


classes = ['greetings', 'goodbye', 'Jewellery', 'name', 'negative feedback', 'positive feedback', 'Specialization', 'Random', 'materialpricegivenweight', 'discounts', 'demands/expectations', 'refund', 'deliverylocation', 'replaceitem', 'deliveryduration', 'timings', 'complaints']
torch.Size([384])


In [7]:
classes = sorted(set(classes))
print(len(classes))
training = []
#initialize the model output with zeros - for one hot encoding
output_empty = [0] * len(classes)

import pickle
classes_file='classes.pkl'
f = open(classes_file,'wb')
pickle.dump(classes,f)
f.close()

17


# New Section

In [8]:
# training = []
data = []
target = []

for document in documents:    #embedding
    output_row = list(output_empty)
    #setting the label for the class
    output_row[classes.index(document[1])] = 1
    data.append(document[0])
    target.append(output_row)
# print(len(training[-1][0]))



In [9]:
# print(len(training[0][0]))

# #trainingdata - embeddings

# # random.shuffle(training)
# print(len(training[0][0]))
# training = np.array(training)
# print(training.shape)
# print(training[:, 0].shape)

# train_x = list(training[:, 0])    # embeddings
# train_y = list(training[:, 1])    # the corresponding classes
# train_x_list = []
# train_y_list = []
# print(len(train_x[0]))


# for x in train_x:
#   train_x_list.append(x.tolist())

# for i in range(len(train_y)):
#   print(train_y[i])

# # print(train_x_list)
# # print(train_y_list)
# train_x = train_x_list
# # train_y = train_y_list

train_x = data
train_y = target
print(np.array(data).shape)
print(np.array(target).shape)

model = Sequential()
model.add(Dense(256, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))                 #dropout for regularizing - train 50% parameters at a single time
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

(106, 384)
(106, 17)


In [12]:
#changed epochs, changed mini-batch size
# your_torch_tensor.numpy()

ai_model = model.fit(train_x, train_y, epochs=500, batch_size=30, verbose=1)
model.save('chatbot_embedding_model.h5', ai_model)
print("AI Chatbot Model Built")

Epoch 1/500
4/4 [==============================] - 0s 5ms/step - loss: 1.1077 - accuracy: 0.5755
Epoch 2/500
4/4 [==============================] - 0s 4ms/step - loss: 1.1870 - accuracy: 0.6132
Epoch 3/500
4/4 [==============================] - 0s 5ms/step - loss: 1.1203 - accuracy: 0.6132
Epoch 4/500
4/4 [==============================] - 0s 5ms/step - loss: 1.0014 - accuracy: 0.6604
Epoch 5/500
4/4 [==============================] - 0s 5ms/step - loss: 1.1431 - accuracy: 0.6038
Epoch 6/500
4/4 [==============================] - 0s 5ms/step - loss: 1.1271 - accuracy: 0.5943
Epoch 7/500
4/4 [==============================] - 0s 5ms/step - loss: 1.1531 - accuracy: 0.5566
Epoch 8/500
4/4 [==============================] - 0s 5ms/step - loss: 1.0970 - accuracy: 0.6038
Epoch 9/500
4/4 [==============================] - 0s 5ms/step - loss: 1.2328 - accuracy: 0.6226
Epoch 10/500
4/4 [==============================] - 0s 5ms/step - loss: 1.1421 - accuracy: 0.5849
Epoch 11/500
4/4 [===========